# <center>Spatial Analysis with Arcpy
<img src="arcpy.png" style="width=300px;">

## 1. Introduction

The objective of this exercise is to create utilitarian Python script for use with ArcGIS. The purpose is to walk through traditional spatial analysis workflow using programming language in stead of using ArcGIS software. The Python script will allow you to perform data management, geoprocessing, and visualization in ArcGIS. This Jupter notebook was created outside of ArcGIS to run the standalone Python script. The major tasks involved to complete this exercise include data cleanup with `pandas`, data analysis with `Arcpy`, and result visualization with `ArcGIS API for Python`.

Data science and the data scientis is all the rage these days. The major taks of a data scientist involves data cleanup, data analysis, modeling/statistics, engineering and prototyping. Just as data science workflow, a spatial analysis workflow involves multiple steps to get to the final results. Besides, with the massive explosion in both the data generated and retained by companies, it is essential for GIS analysts to equip skills for handling high volumne of data. Python, R, and SQL are very popular amongst the data science community. Hence, it is important for us, GISers, to become familiar with the languages to enable us to work more effeciently and elegantly. 

The data used for this exercise includes 
- (1) A csv file of 911 calls in Baltimore from 2015-2018
- (2) Baltimore district shapefile
- (3) TIGER/Line shapefile of Maryland census tract 

## 2. Workflow
### 2.1 Data Munging

Before we get started, try to open **"911_Police_calls_for_Service.csv"** in Excel and see what happens? Yes, you saw an error message prompted out and told you Excel was overloaded. Why? Because Excel is only able to handle one million of rows at a time. 

Now, could you come up with any solution to check and clean up this data instead of using Excel? You may have guessed that it can be opened and read using programming language and you are correct. To handle this cumbersome file, we will use the python library - `pandas`. 

Now, Let's look how `pandas` can help us carry out data munging. 

#### 2.1.1 Read the File

We first import `pandas` and read the file into dataframe.
> `pandas` dataframe is two-dimensional size-mutable, potentially heterogeneous tabular data structure with labeled axes (rows and columns). It consists of three principal components, the *data*, *rows*, and *columns*. It enables us to view, manage, and modify the data much easier. 

In [1]:
import pandas as pd

# Read the csv file into dataframe
df = pd.read_csv("911_Police_Calls_for_Service.csv")
# Display the first five rows in the dataframe
df.head()

,recordId,callDateTime,priority,district,description,callNumber,incidentLocation,location
0,1423624,05/04/2016 09:58:00 PM,High,ND,SILENT ALARM,P161253035,400 WINSTON AV,"400 WINSTON AV\nBALTIMORE, MD\n(39.349792, -76..."
1,1402097,04/27/2016 03:57:00 PM,Medium,SW,911/HANGUP,P161182081,1400 BRADDISH AV,"1400 BRADDISH AV\nBALTIMORE, MD\n(39.303941, -..."
2,1420176,05/03/2016 06:40:00 PM,Medium,ED,DISORDERLY,P161242705,200 E NORTH AV,"200 E NORTH AV\nBALTIMORE, MD\n(39.311294, -76..."
3,1423653,05/04/2016 10:10:00 PM,Medium,NE,911/NO VOICE,P161253068,2500-1 HARFORD RD,"2500 1 HARFORD RD\nBALTIMORE, MD\n(39.316763, ..."
4,1417949,05/03/2016 12:29:00 AM,Non-Emergency,SD,Private Tow,P161240063,100 W PATAPSCO AV,"100 W PATAPSCO AV\nBALTIMORE, MD\n(39.239215, ..."


Let's see how many rows and columns are there in the dataset and display columns' names.

In [2]:
# Display the number of columns and rows inside the dataframe
print("Total rows: {0} \nTotal columns: {1}".format(df.shape[0], df.shape[1]))
print("Columns: {}".format(df.columns))

Total rows: 4078343 
Total columns: 8
Columns: Index(['recordId', 'callDateTime', 'priority', 'district', 'description',
       'callNumber', 'incidentLocation', 'location'],
      dtype='object')


As you can see that. Ther are over four millions rows!! No wonder Excel crashed. In fact, you will only ongoingly face this such large file in this big data era. 

Here let's consider what kinds of cleanup are required for our later analysis. 
- *Remove unnecessary columns containing inapplicable information*: Not all the columns contain information we need for analysis. 
- *Remove records(rows) containing empty values*: The file might have blank cells, and this will be deonted by *"NaN"* in `pandas`. 
- *Derive year, month, and weekday from the "callDateTime" field*: In this exercise, we are interested in which month and weekday has the highest number of  emergency 911 calls. Since all this information are combined together, we need to parse or generate them by taking advantage of `pandas` functionality. 
- *Slice records(rows) in year of 2018*: For this exercise, we are only looking at the 911 calls in 2018; however, there is three years span in this data.
- *Extract coordinates information from the "location" field*: For the later analysis, we will need to plot 911 calls to the point feature layer. However, coordinates information were recorded with location address and this format is not readable. Hence, we need to slice them out.

#### 2.2.2 Remove Unnecessary Columns and Empty Rows

In [3]:
# Slice the dataframe using desired column names 
df1 = df[["priority", "district", "location", "callDateTime"]]
# Check the result
df1.head(1)

,priority,district,location,callDateTime
0,High,ND,"400 WINSTON AV\nBALTIMORE, MD\n(39.349792, -76...",05/04/2016 09:58:00 PM


Let's display all the unique values in "priority" field to see if there is a typo, such as "Emergency" was carelessly input as "Emerg".

In [4]:
# Display unique values
print(df1.priority.unique())

['High' 'Medium' 'Non-Emergency' 'Low' 'Emergency' 'Out of Service' nan]


Great ! It looks like there is no typo in the "priority" field. However, you can see that there are empty cells in this field. Let's remove them. 

In [5]:
# Drop the rows containing non value
df2 = df1.dropna()
# Display the remaining rows
print("Total rows: {0} ".format(df2.shape[0]))

Total rows: 4007395 


Here, you can see that the number of rows reduce from 4,078,343 to 4,007,395.

#### 2.3.3 Derive Year, Month, and Weekday and Slice Rows in 2018

We first slice date from the "callDateTime" field.

In [6]:
# Slice date out and create a new field to store it
df2["Date"] = df2.callDateTime.str.slice(0,10)
# Check the result
df2.head(1)

C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,priority,district,location,callDateTime,Date
0,High,ND,"400 WINSTON AV\nBALTIMORE, MD\n(39.349792, -76...",05/04/2016 09:58:00 PM,05/04/2016


Next, we slice year and month from the "Date" field.

In [7]:
# Slice month out and create a new field to store it
df2["Month"] = df2.Date.str.slice(0, 2)
# Slice year out and create a new field to store it
df2["Year"] = df2.Date.str.slice(-4,)
# Check the result
df2.head(1)

C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,priority,district,location,callDateTime,Date,Month,Year
0,High,ND,"400 WINSTON AV\nBALTIMORE, MD\n(39.349792, -76...",05/04/2016 09:58:00 PM,05/04/2016,05,2016


Next, we slice the rows in the year of 2018 and see how many rows remain.

In [8]:
# Slice rows in year of 2018
df3 = df2[df2.Year == "2018"]
# Check the result
print(df3.Year.unique())
# Display the remaining rows
print("Total rows: {0} ".format(df3.shape[0]))

['2018']
Total rows: 952530 


Now, let's convert "Date" filed into the `pandas` "datatime" data type in order to derive the weekday information.

In [9]:
# Convert to the datatime data type
df3["Date"] = pd.to_datetime(df3.Date)
# Check the data tyoe of "Date" Field
print(df3.Date.dtypes)

datetime64[ns]


C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


"datetime" format is a special data type in `pandas`, it provides lots of benefits to manage date or time. In the cell below, we are going to acquire weekday information based on it.

In [10]:
# Acquire weekay information based on "Date" field
df3["Weekday"] = df3.Date.dt.weekday
# Check the result
df3.head(1)

C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,priority,district,location,callDateTime,Date,Month,Year,Weekday
6,High,SE,"N PULASKI HY\nBALTIMORE, MD",01/20/2018 10:20:00 AM,2018-01-20,01,2018,5


#### 2.4.4 Extract Coordinates Information

Let's take a look how coordinates information were recorded in the "location" field.

In [11]:
# Print the values in "location field"
df3.location

6                                N PULASKI HY\nBALTIMORE, MD
7                                W N NORTH AV\nBALTIMORE, MD
3603                           E N FAYETTE ST\nBALTIMORE, MD
3608                           100 W EAGER ST\nBALTIMORE, MD
3615                           E N FAYETTE ST\nBALTIMORE, MD
3623                       2000 BLK BELAIR RD\nBALTIMORE, MD
3633                           2300 BANGER ST\nBALTIMORE, MD
3637                               S CAREY ST\nBALTIMORE, MD
3639                        0 BLK S GILMOR ST\nBALTIMORE, MD
3641                             FREDERICK AV\nBALTIMORE, MD
3642                            E N BIDDLE ST\nBALTIMORE, MD
3643                             300 BLOOM ST\nBALTIMORE, MD
3644                                1200 RAMP\nBALTIMORE, MD
3645                         4700 N ROGERS AV\nBALTIMORE, MD
3648                           N MCELDERRY ST\nBALTIMORE, MD
3649                               N E GAY ST\nBALTIMORE, MD
3650                    

Here, you can see some cells have coordinates information but some do not have. In the above code, we use `Series.str.slice` to extract date information; however, we cannot use it here as we may acquire substring of address instead of coordinates. To solve this problem, we will use `regex` module to extract coordinates information. 

**Regular expression** or `regex` is a special text string for describing a search pattern. We will first create a "patter object" and if a string matches the pattern object, it will return a match object.

`pandas` has a function to extract groups from the first match of `regex` expression pattern object. Now, let's see how it works.

In [12]:
import re

# Create the pattern object
p = re.compile(r".*\n.*\n\((.*),(.*)\)")

# Return a new dataframe with columns of capture groups
df4 = df3.location.str.extract(p, expand = False)

# Check the result
df4

,0,1
6,NaN,NaN
7,NaN,NaN
3603,NaN,NaN
3608,NaN,NaN
3615,NaN,NaN
3623,NaN,NaN
3633,NaN,NaN
3637,NaN,NaN
3639,NaN,NaN
3641,NaN,NaN


Here, you can see that a new dataframe was created and two columns were added to it. The column "0" represents the first match group, which is "latitude", the column "1' represents the second match group, which is "longitude". If the cells in "location" field do not contain coordinates, the "NaN" value will be returned.

Next,let's rename the columns to make sense of them, join it back to the original dataframe, and remove rows containing empty values.

In [13]:
# Rename the columns' names
df4.rename(columns = {0: "Latitude", 1 : "Longitude"}, inplace = True)
# Check the result
df4.head()

,Latitude,Longitude
6,NaN,NaN
7,NaN,NaN
3603,NaN,NaN
3608,NaN,NaN
3615,NaN,NaN


In [14]:
# Join columns from df4 to df3
df5 = df3.join(df4)
# Check the result
df5.head()

,priority,district,location,callDateTime,Date,Month,Year,Weekday,Latitude,Longitude
6,High,SE,"N PULASKI HY\nBALTIMORE, MD",01/20/2018 10:20:00 AM,2018-01-20,01,2018,5,NaN,NaN
7,High,SW,"W N NORTH AV\nBALTIMORE, MD",01/20/2018 08:37:00 AM,2018-01-20,01,2018,5,NaN,NaN
3603,Medium,CD,"E N FAYETTE ST\nBALTIMORE, MD",01/01/2018 01:56:00 AM,2018-01-01,01,2018,0,NaN,NaN
3608,Low,CD,"100 W EAGER ST\nBALTIMORE, MD",01/05/2018 02:16:00 PM,2018-01-05,01,2018,4,NaN,NaN
3615,Medium,CD,"E N FAYETTE ST\nBALTIMORE, MD",01/09/2018 06:26:00 PM,2018-01-09,01,2018,1,NaN,NaN


In [15]:
# Drop the rows containing NaN
df5.dropna(inplace = True)
# Check the result
df5.head()

,priority,district,location,callDateTime,Date,Month,Year,Weekday,Latitude,Longitude
42962,Medium,NW,"3600 FORDS LN\nBALTIMORE, MD\n(39.359715, -76....",01/13/2018 07:45:00 AM,2018-01-13,01,2018,5,39.359715,-76.697044
42963,Low,NW,"3600 PLATEAU AV\nBALTIMORE, MD\n(39.333044, -7...",01/23/2018 06:34:00 PM,2018-01-23,01,2018,1,39.333044,-76.699367
42964,Medium,CD,"300 W PRESTON ST\nBALTIMORE, MD\n(39.301803, -...",01/11/2018 03:43:00 PM,2018-01-11,01,2018,3,39.301803,-76.622675
42965,High,SD,"1000 W PATAPSCO AV\nBALTIMORE, MD\n(39.246726,...",01/09/2018 08:38:00 AM,2018-01-09,01,2018,1,39.246726,-76.636095
42966,Medium,NW,"5500 JONQUIL AV\nBALTIMORE, MD\n(39.349911, -7...",01/15/2018 12:18:00 AM,2018-01-15,01,2018,0,39.349911,-76.686644


Here, we are going to display the data type of each column and convert columns' data types to the desired format. 

We would like "Month" and "Year" in integer and "Latitude" and "Longitude" in float.

In [16]:
# Display columns' data types
df5.dtypes

priority                object
district                object
location                object
callDateTime            object
Date            datetime64[ns]
Month                   object
Year                    object
Weekday                  int64
Latitude                object
Longitude               object
dtype: object

In [17]:
# Convert to desired data types
df6 = df5.astype({"Month":int, "Year":int,
                  "Latitude":float, "Longitude":float})
# Check the result
df6.dtypes

priority                object
district                object
location                object
callDateTime            object
Date            datetime64[ns]
Month                    int32
Year                     int32
Weekday                  int64
Latitude               float64
Longitude              float64
dtype: object

Excellent!! Now the data is clean and allows us to do the analysis. Let's see how many rows are remained and export this clean data to a csv file for later usage. 

In [18]:
# Display the remaining rows
print("Total rows: {0} ".format(df6.shape[0]))

Total rows: 701856 


In [19]:
# Save as a new csv file
df6.to_csv("911_calls_in_2018.csv", index = False)

Recaping from the above cells, you took advantage of `pandas` module to read and modify your data and `regex` module to extract coordinates information in an efficient manner. The file size reduced from about 530 MB to 90 MB, this will allow ArcGIS software read and proceed it easier and faster. However, in the age of big data, this file size is quite small. What if you want to read a file with larger size in ArcGIS software? Even if ArcGIS can read it, it might still be strenous for ArrGIS to process. Therefore, it is better off to keep working with programming language. And this is where `ArcPy` comes into play. 

### 2.2 Data Analysis

In the following cells, we will take advantage of `ArcPy` Python site package to perform data analysis. `ArcPy` provides a rich modules and functions to carry out geographic data analysis, data conversion, data management, and map automation. 

For this exercise, we are going to perform two types of geospatial analysis.
> 1. Spatial Join Analysis
2. Hotspot Analysis 

#### 2.1.1 Set Up Working Environment

In the cell below, we first import `ArcPy` module and then set up our wokring environment. The working environment is where the analysis output will be stored and managed, we typically usa a file geodatabase as the working environment. We can also set the properties of the working environment, such as allow file overwritting and coordinate system of analysis output. These settings can help us work faster and easier. 

In [1]:
import arcpy

# Set up current working directory
current_dir = r"C:\Users\user\Documents\ArcGIS\Projects"
# Set up your workspace
arcpy.env.workspace = current_dir + r"\Assignment 4\Assignment 4.gdb"
# Allow file overwritting
arcpy.env.overwriteOutput = True
# Set the workspace outputCoordinateSystem environment
# This setting will automatically convert your analysis output to this projected system
arcpy.env.outputCoordinateSystem = arcpy.SpatialReference(6487)

#### 2.2.2 Data Preparation

The data used for analysis includes **911 calls**, **TIGER/Line Maryland census tract**, and **Baltimore district**. To use these data, we need to perform some tasks first. 

- *Plot 911 calls* to point feature class: The 911 calls data is now stored in the csv file we created above. To use it for analysis, we need to create a point feature class of 911 calls data.
- *Select out the study area*: Since TIGHER/Line census tract can only be downloaded by state, we need to perform selection analysis to select our study area- Baltimore county. 
- *Clip 911 calls to the study area*: There might be some typos of coordinate values that lead to the 911 calls fall oustide the study area, so we will perform clip analysis to ensure all the 911 calls are wihtin the study area.

##### 2.2.2.1 Plot 911 Calls

In [22]:
# Set path to input file and create output file name
# The input file is the csv file we created above
input_file = current_dir + r"\Assignment 4 _script\Stats19-Data1979-2004\911_calls_in_2018.csv"
output_file = "Calls_2018"

# Run the "XY Table To Point" tool
arcpy.management.XYTableToPoint(input_file, output_file, 
                                x_field = "Longitude", y_field = "Latitude")

# Check the coordinate system
print(arcpy.Describe(output_file).Spatialreference.name)

NAD_1983_2011_StatePlane_Maryland_FIPS_1900


##### 2.2.2.2 Select Out the Study Area

To convert **TIGER/Line Maryland census tract shapefile** to a polygon feature class, let's first look the fields' names in the census tract shapefile.  

In [2]:
# Define file's name
input_file = current_dir + r"\Assignment 4\Data\Census_Tract\tl_2018_24_tract.shp"

# Read the fields containing in the shapefile and diplay their names using the "ListFields" function
fields = arcpy.ListFields(input_file)
for field in fields:
    print(field.name)

FID
Shape
STATEFP
COUNTYFP
TRACTCE
GEOID
NAME
NAMELSAD
MTFCC
FUNCSTAT
ALAND
AWATER
INTPTLAT
INTPTLON


In [3]:
# Read the value of the "COUNTYFP" field to verify the fip code scheme
rows = arcpy.SearchCursor(input_file, fields = "COUNTYFP")
for row in rows:
    print("COUNTYFP:{0}".format(row.getValue("COUNTYFP")))
    break

COUNTYFP:033


In [4]:
# Set parameters
path = current_dir + r"\Assignment 4\Assignment 4.gdb"
output_feature = "Baltimore_CT"
# Run the "Feature Class to Feature Class" tool and only extract the features where their "COUNTYFP" matches Baltimore's Fip
arcpy.FeatureClassToFeatureClass_conversion(input_file, path, output_feature, 
                                            "COUNTYFP='510'")
# Check the coordinate system
print(arcpy.Describe(output_feature).SpatialReference.Name)

NAD_1983_2011_StatePlane_Maryland_FIPS_1900


Convert **Baltimore district shapefile** to a polygon feature class

In [5]:
# Set parameters
input_file = current_dir + r"\Assignment 4\Data\Legislative_Districts\Legislative_Districts.shp"
path = current_dir + r"\Assignment 4\Assignment 4.gdb"
output_feature = "Baltimore_district"

# Run the "Feature Class to Feature Class" tool
arcpy.FeatureClassToFeatureClass_conversion(input_file, path, output_feature)

# Check the coordinate system
print(arcpy.Describe(output_feature).SpatialReference.Name)

NAD_1983_2011_StatePlane_Maryland_FIPS_1900


##### 2.2.2.3 Clip 911 Calls to the Study Area

In [6]:
# Set parameters
input_feature = "Calls_2018"
output_feature = "Baltimore_calls_2018"
cutter = "Baltimore_CT"

# Run the "Clip" tool
arcpy.Clip_analysis(input_feature, cutter, output_feature)

# Check the coordinate system
print(arcpy.Describe(output_feature).SpatialReference.Name)

NAD_1983_2011_StatePlane_Maryland_FIPS_1900


### 2.3 Perform Spatial Join Analysis

In this exercise, we are interested in answering two questions. The questions are as the following:
> 1. Identify which district has the highest number of 911 calls.  
2. Identify which month and weekday has the highest number of 911 calls in each district.

#### 2.3.1 Identify which district has the highest number of 911 calls

To answer this question, we have to spatial join the "911 calls" point feature class into the "Baltimore" polygon feature class. Then, we will see how many calls are there in each district and display the name of district having the highest number and its total number.

In [7]:
# Define files' names
target_feature = "Baltimore_district"
join_feature = "Baltimore_calls_2018"
output_feature = "Total_calls_district"

# Run the sptail join tool to identify total accident number in each county
arcpy.SpatialJoin_analysis(target_feature, join_feature, 
                           output_feature, match_option = "COMPLETELY_CONTAINS")

<Result 'C:\\Users\\user\\Documents\\ArcGIS\\Projects\\Assignment 4\\Assignment 4.gdb\\Total_calls_district'>

In [8]:
# Print out all the field names and their type in the output feature
# Use the "ListFields" function
fields = arcpy.ListFields(output_feature)

for field in fields:
    print(field.name,":", field.type)

OBJECTID_1 : OID
Shape : Geometry
Join_Count : Integer
TARGET_FID : Integer
OBJECTID : Integer
AREA_NAME : String
AREA_NMBR : Integer
URL : String
ShapeSTAre : Double
priority : String
district : String
location : String
callDateTime : String
Date : Date
Month : Integer
Year : Integer
Weekday : Integer
Latitude : Double
Longitude : Double
Shape_Length : Double
Shape_Area : Double


In [9]:
# Using search cursor to iterate through the feature class and
# display the name and total accident number of the county has the highest total accidents 
rows = arcpy.SearchCursor(output_feature, fields = "Join_Count; AREA_NAME", 
                          sort_fields = "Join_Count D")
for row in rows:
    print("'{0}' {1} {2}".format(
        row.getValue("AREA_NAME"),
        "has the highest number of 911 calls in 2018, the total count is",
        row.getValue("Join_Count")))
    break

'State Legislative District 40' has the highest number of 911 calls in 2018, the total count is 163423


#### 2.3.2 Identify which month and weekday had the highest number of 911 calls in each district.

To answer this question, we will include "Weekday", and "Month" information to our output table so that we can see which month and weekday had the highest number of 911 calls in each district.

In [10]:
# Define files' names
target_feature = "Baltimore_district"
join_feature = "Baltimore_calls_2018"
output_feature = "Types_calls_district"

In [11]:
# Creat the required FieldMap and FieldMappings objects
fm_month = arcpy.FieldMap()
fm_district = arcpy.FieldMap()
fm_weekday = arcpy.FieldMap()
fms = arcpy.FieldMappings()

# Get each field name from their original file
month = "MONTH"
district = "AREA_NAME"
weekday = "Weekday"

# Add fields to their corresponding FieldMap objects
fm_month.addInputField(join_feature, month)
fm_weekday.addInputField(join_feature, weekday)
fm_district.addInputField(target_feature, district)

# Get the output field's properties as a field object
# Rename the field and pass the updated field object back into the field map
district_name = fm_district.outputField
district_name.name = "District"
district_name.aliasName = "District"
fm_district.outputField = district_name

# Set merge rule to FieldMaps
fm_month.mergeRule = "Mode"
fm_weekday.mergeRule = "Mode"

# Add the FieldMap objects to the FieldMappings object
fms.addFieldMap(fm_district)
fms.addFieldMap(fm_weekday)
fms.addFieldMap(fm_month)

In [12]:
# Run the "Spatial Join" tool with Fieldmappings object
arcpy.SpatialJoin_analysis(target_feature, join_feature, 
                           output_feature, match_option = "COMPLETELY_CONTAINS", 
                           field_mapping = fms)

<Result 'C:\\Users\\user\\Documents\\ArcGIS\\Projects\\Assignment 4\\Assignment 4.gdb\\Types_calls_district'>

In [13]:
# Display fields' names of output feature
for field in arcpy.ListFields(output_feature):
    print(field.name)

OBJECTID
Shape
Join_Count
TARGET_FID
District
Weekday
Month
Shape_Length
Shape_Area


In [14]:
# Using the "SearchCursor" function to iterate through the feature class and
# display county name and the month having the highest occurence of accidents in this county 
rows = arcpy.SearchCursor(output_feature, 
                          fields = "District; Month")
for row in rows:
    print("{0} {1} {2} '{3}'.".format(
        "Month:",
        row.getValue("Month"),
        "has the highest number of 911 calls in",
        row.getValue("District")))

Month: 5 has the highest number of 911 calls in 'State Legislative District 40'.
Month: 5 has the highest number of 911 calls in 'State Legislative District 41'.
Month: 5 has the highest number of 911 calls in 'State Legislative District 43'.
Month: 12 has the highest number of 911 calls in 'State Legislative District 44A'.
Month: 5 has the highest number of 911 calls in 'State Legislative District 45'.
Month: 8 has the highest number of 911 calls in 'State Legislative District 46'.


In [15]:
# Using the "SearchCursor" function to iterate through the feature class and
# display county name and the weekday having the highest occurence of accidents in this county 
rows = arcpy.SearchCursor(output_feature, 
                          fields = "District; Weekday")
for row in rows:
    print("{0} {1} {2} '{3}'.".format(
        "Weekday:",
        row.getValue("Weekday"),
        "has the highest number of 911 calls in",
        row.getValue("District")))

Weekday: 4 has the highest number of 911 calls in 'State Legislative District 40'.
Weekday: 4 has the highest number of 911 calls in 'State Legislative District 41'.
Weekday: 4 has the highest number of 911 calls in 'State Legislative District 43'.
Weekday: 4 has the highest number of 911 calls in 'State Legislative District 44A'.
Weekday: 4 has the highest number of 911 calls in 'State Legislative District 45'.
Weekday: 4 has the highest number of 911 calls in 'State Legislative District 46'.


### 2.4. Perform Hotspot Analysis

In the following cells, we are going to perform hotspot analysis to create the hotspot map of high priority 911 calls in Baltimore.

#### 2.4.1 Select out high-priority 911 Calls

In [16]:
# Define file's name
input_feature = "Baltimore_calls_2018"

In [19]:
# Define SQL expression
where_clause = "priority = 'High'"

# Run the "Select Layer by Attribute tool" to select out accidents with deaths 
High_priority_calls = arcpy.SelectLayerByAttribute_management(input_feature, "NEW_SELECTION", 
                                                              where_clause)

# Export to a new feature class using the "Copy Features" tool
arcpy.CopyFeatures_management(High_priority_calls, 'High_priority_calls')

<Result 'C:\\Users\\user\\Documents\\ArcGIS\\Projects\\Assignment 4\\Assignment 4.gdb\\High_priority_calls'>

#### 2.4.2 Using census tract as the basic unit

In [20]:
# Define parameters
input_feature = "High_priority_calls"
output_feature = "Hotspot_hp_calls_CT"
Aggregating_Polygon = "Baltimore_CT"
# Define output folder 
output_folder = current_dir + r"\Assignment 4\Data"

In [21]:
# Run the "Optimized hotspot analysis" tool with census tract as aggregating polygon
arcpy.OptimizedHotSpotAnalysis_stats(input_feature, output_feature, 
                                     "#", "COUNT_INCIDENTS_WITHIN_AGGREGATION_POLYGONS", 
                                     "#", Aggregating_Polygon, "#", "#", "#")

# Save the output file as a shapefile
arcpy.FeatureClassToShapefile_conversion(output_feature, output_folder)

<Result 'C:\\Users\\user\\Documents\\ArcGIS\\Projects\\Assignment 4\\Data'>

#### 2.4.3 Using fishnet grid as the basic unit

In [22]:
# Define outout feature class name
output_feature = "Hotspot_hp_calls_Fishnetgrid"

In [23]:
# Run the "Optimized hotspot analysis" tool with fishnet grid as aggregating polygon
arcpy.OptimizedHotSpotAnalysis_stats(input_feature, 
                                     output_feature, 
                                     "#", "COUNT_INCIDENTS_WITHIN_FISHNET_POLYGONS", 
                                     "#", "#", "#", "#", "#")

# Save the output file as a shapefile
arcpy.FeatureClassToShapefile_conversion(output_feature, output_folder)

<Result 'C:\\Users\\user\\Documents\\ArcGIS\\Projects\\Assignment 4\\Data'>

Now, we have finished hotspot analysis and next we are about to visualize the results using function provided by `ArcGIS API for Python`.

### 2.5 Result Visualization

`ArcGIS API for Python` is a Python library for working with maps and geospatial data, powered by web GIS. It not only provides a rich set of functions for geospatial analysis but also allows us to manage and organize our GIS content.

For more information about `ArcGIS API for Python`, please refer to this [website](https://developers.arcgis.com/python/)

The procedure to visualize the results involves two steps:
- Upload datasets to your GIS content and publish them as feature layer.
- Create a basemap and add the feature layers on it.

#### 2.5.1 Upload datasets

We first import "GIS" Object from the "gis" module within `ArcGIS API for Python`. Then we create a GIS object by passing in login credential. 

In [24]:
# Import GIS object
from arcgis.gis import GIS
# Import display function from IPython module in order to display GIS content later
from IPython.display import display
# Create a GIS object
gis = GIS("Pro")

In [25]:
# Define the properties of the file that is going to be uploaded
CT_hot_properties = {
    "title" : "Hotspot_hp_calls_CT",
    "type" : "Shapefile",
    "tags" : "Hotspot, High Priority Calls, Baltimore",
    "description":"{0}{1}{2}".format("This hotspot map was created for course usage.", 
                                     "This map displays hotspot of high priority 911 calls in Baltimore.",
                                     "Census tracts are used as the geographic unit for hotspot analysis.")
}             

# Define the path of the file
upload_file = current_dir + r"\Assignment 4\Data\Hotspot_hp_calls_CT.zip"

# Add the file to the GIS content, which is your ArcGIS Online content
CT_hotspot = gis.content.add(item_properties = CT_hot_properties, data = upload_file)

# Publish the file to a feature layer
CT_hotspot_item = CT_hotspot.publish()

In [26]:
# Define the properties of the file that is going to be uploaded
FG_hot_properties = {
    "title" : "Hotspot_hp_calls_FG",
    "type" : "Shapefile",
    "tags" : "Hotspot, High Priority Calls, Baltimore",
    "description":"{0}{1}{2}".format("This hotspot map was created for course usage.", 
                                     "This map displays hotspot of high priority 911 calls in Baltimore.",
                                     "Fishnet grids are used as the geographic unit for hotspot analysis.")
}             

# Define the path of the file
upload_file = current_dir + r"\Assignment 4\Data\Hotspot_hp_calls_Fishnetgrid.zip"

# Add the file to the GIS content, which is your ArcGIS Online content
FG_hotspot = gis.content.add(item_properties = FG_hot_properties, data = upload_file)

# Publish the file to a feature layer
FG_hotspot_item = FG_hotspot.publish()

Here, we have finished uploading and publishing the datasets. In the cell below, let's search them in the GIS content. 

In [27]:
# Search feature layers contain "Hotspot_hp_calls"
search_result = gis.content.search(query= "Hotspot_hp_calls", 
                                   item_type = "Feature Layer")

# Display them
for layer in search_result:
    display(layer)

<Item title:"Hotspot_hp_calls_FG" type:Feature Layer Collection owner:lin00297_UMN>

<Item title:"Hotspot_hp_calls_CT" type:Feature Layer Collection owner:lin00297_UMN>

Note that both feature layers are "Feature Layer Collection" item. Since this item is a Feature Layer Collection, accessing the `layers` property will give us a list of `FeatureLayer` objects

In [28]:
# Read as feature layer instead feature layer collection
# This is useful if you want to read the properties of your files
hotspot_hp_calls_FG = search_result[0].layers
hotspot_hp_calls_CT = search_result[1].layers
print(hotspot_hp_calls_FG,"\n",hotspot_hp_calls_CT)

[<FeatureLayer url:"https://services.arcgis.com/8df8p0NlLFEShl0r/arcgis/rest/services/Hotspot_hp_calls_Fishnetgrid/FeatureServer/0">] 
 [<FeatureLayer url:"https://services.arcgis.com/8df8p0NlLFEShl0r/arcgis/rest/services/Hotspot_hp_calls_CT/FeatureServer/0">]


Next, let us take a closer look at the properties of the files

In [29]:
# Read the extent of the "hotspot_hp_calls_FG"
hotspot_hp_calls_FG[0].properties.extent

{
  "xmin": -8539554.469651511,
  "ymin": 4750219.9411982335,
  "xmax": -8519137.288212582,
  "ymax": 4775178.12296056,
  "spatialReference": {
    "wkid": 102100,
    "latestWkid": 3857
  }
}

The properties field on a `FeatureLayer` object provides a dictionary representation of all its properties. However you can access individual properties as fields as well.

Now, we want to know the names of fields present in these layers because we want to classify our map based on the desired fields. To check the names of fields, we can call the `fields` property:

In [30]:
for field in hotspot_hp_calls_FG[0].properties.fields:
    print(field["name"])

FID
SOURCE_ID
JOIN_COUNT
Shape_Leng
Shape_Area
GiZScore
GiPValue
NNeighbors
Gi_Bin
Shape__Area
Shape__Length


In [31]:
for field in hotspot_hp_calls_CT[0].properties.fields:
    print(field["name"])

FID
SOURCE_ID
Join_Count
Shape_Leng
Shape_Area
GiZScore
GiPValue
NNeighbors
Gi_Bin
Shape__Area
Shape__Length


Great, now we have all the names of fields and we will use "Gi_Bin" as our classified field.

####  2.5.2 Create a basemap and add the feature layers on it

`gis` module provides opportunity to create a map widget centered at the declared location by using `map` function.

In [32]:
# Create a map widget centered at Baltimore as basemap
map_for_FG = gis.map(location="Baltimore")
map_for_FG

MapView(layout=Layout(height='400px', width='100%'))

In [33]:
# Create a map widget centered at Baltimore as basemap.
map_for_CT = gis.map(location="Baltimore")
map_for_CT

MapView(layout=Layout(height='400px', width='100%'))

This map widge also provides us a list of basemap style options. We can use `basemap` method to see all the available options.

In [34]:
# Display available basemap styles
map_for_FG.basemaps

['dark-gray',
 'dark-gray-vector',
 'gray',
 'gray-vector',
 'hybrid',
 'national-geographic',
 'oceans',
 'osm',
 'satellite',
 'streets',
 'streets-navigation-vector',
 'streets-night-vector',
 'streets-relief-vector',
 'streets-vector',
 'terrain',
 'topo',
 'topo-vector']

Let's select "dark-gray-vector" as our basemap style

In [35]:
# Change basemap style
map_for_FG.basemap='dark-gray-vector' 
map_for_CT.basemap='dark-gray-vector'

Now, lets add feature layer to the basemap

In [36]:
# Add the feature layer onto the basemap, note that we are using feature layer collection as the parameter 
map_for_FG.add_layer(search_result[0], 
                     {"renderer":"ClassedColorRenderer","field_name":"Gi_Bin", "opacity":0.7})
# Add the legend
map_for_FG.legend = True

In [37]:
# Add the feature layer onto the basemap, note that we are using feature layer collection as the parameter 
map_for_CT.add_layer(hotspot_hp_calls_CT, 
                     {"renderer":"ClassedColorRenderer","field_name":"Gi_Bin", "opacity":0.7})
# Add the legend
map_for_CT.legend = True

You can see feature layers are added onto the baseamaps. 

## 3. Conclusion

In this exercise, we walked through the common spatial workflow to finish our analysis using Python script with ArcGIS. You can see the strength of programming language in helping read large file, manage data, and analyze it.

For more question about this exercise, please email to lin00297@umn.edu.